## Question-2 Part2- English to Spanish

### Loading the Data

In [ ]:
from torch.optim import lr_scheduler
from __future__ import print_function, division
import torch.optim as optim
%matplotlib inline
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision
import pickle
import os
from torchvision import datasets, models, transforms
import copy
import time
import numpy as np
from google.colab import drive
import heapq

In [ ]:
drive.mount('/content/drive')
%cd /content/drive/MyDrive/IDS576_Assignment_3
cuda = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Mounted at /content/drive
/content/drive/MyDrive/IDS576_Assignment_3


In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import random
import re
import unicodedata
import string
import torch.nn.functional as F
cuda = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SOS_tkn = 0
EOS_tkn = 1

In [ ]:
class Lang:
  def __init__(self, name):
      self.name = name
      self.wordToIndex = {}
      self.word2count = {}
      self.index2word = {0: "SOS", 1: "EOS"}
      self.min_words = 2

  def addSentence(self, sentence):
    for word in sentence.split(' '):
      self.addWord(word)

  def addWord(self, word):
    if word not in self.wordToIndex:
      self.wordToIndex[word] = self.min_words
      self.word2count[word] = 1
      self.index2word[self.min_words] = word
      self.min_words += 1
    else:
      self.word2count[word] += 1

In [ ]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.wordToIndex = {}
        self.wordToCount = {}
        self.indexToword = {0: "SOS", 1: "EOS"}
        self.min_words = 2
        self.words=[]

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.wordToIndex:
            self.wordToIndex[word] = self.min_words
            self.wordToCount[word] = 1
            self.indexToword[self.min_words] = word
            self.words.append(word)
            self.min_words += 1
        else:
            self.wordToCount[word] += 1
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [ ]:
def readLangs(lng_1, lng_2, reverse=False):
    print("Reading data lines...")

    lines = open('/content/drive/MyDrive/IDS576_Assignment_3/spa-eng/spa.txt' , encoding='utf-8').\
        read().strip().split('\n')

    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    l=[]
    for s in pairs:
      l.append(s[0:2])
    pairs = l
    pairs


    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        inp_lang , op_lang= Lang(lng_2) ,Lang(lng_1)

    else:
        inp_lang, op_lang = Lang(lng_1) , Lang(lng_2)

    return inp_lang, op_lang, pairs

MX_LENGTH = 10

eng_prefixes = ("i am ", "i m ", "he is", "he s ", "she is", "she s ","you are", "you re ", "we are", "we re ", "they are", "they re ")

def filterPair(p):
    return len(p[0].split(' ')) < MX_LENGTH and \
        len(p[1].split(' ')) < MX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
def prepareData(lng_1, lng_2, reverse=False):
    inp_lang, op_lang, pairs = readLangs(lng_1, lng_2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting the words...")
    for pair in pairs:
        inp_lang.addSentence(pair[0])
        op_lang.addSentence(pair[1])
    print("Words Counted:")
    print(inp_lang.name, inp_lang.min_words)
    print(op_lang.name, op_lang.min_words)
    return inp_lang, op_lang, pairs

In [ ]:
inp_lang, op_lang, pairs = prepareData('eng', 'spa')
print(random.choice(pairs))

Reading data lines...
Read 118964 sentence pairs
Trimmed to 90751 sentence pairs
Counting the words...
Words Counted:
eng 10763
spa 20221
['that s not a question .', 'no es una pregunta .']


### Loading glove.6b.100d.txt File

In [ ]:
emmbed_dict = {}
with open('/content/drive/MyDrive/IDS576_Assignment_3/glove.6B.100d.txt','r') as f:
  for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:],'float32')
    emmbed_dict[word]=vector


def glove_embeding (glove_word):
  try:
    gloveVector = emmbed_dict[glove_word]
  except KeyError:
    gloveVector = np.random.normal(scale=0.6, size=(100, ))
  return gloveVector

matrix_len = inp_lang.min_words
weightsMatrix = np.zeros((matrix_len, 100))
words_found = 0

for i, word in enumerate(pairs[0]):
    try:
        weightsMatrix[i] = emmbed_dict[word]
        words_found += 1
    except KeyError:
        weightsMatrix[i] = np.random.normal(scale=0.6, size=(100, ))

In [ ]:
def create_emb_layer(weightsMatrix, non_trainable=False):
    numEmbeddings, embeddingDim = weightsMatrix.shape
    emb_layer = nn.Embedding(numEmbeddings, embeddingDim)
    emb_layer.load_state_dict({'weight':torch.tensor(weightsMatrix, dtype=torch.long, device=cuda) })
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, numEmbeddings, embeddingDim

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hdnSize):
        super(EncoderRNN, self).__init__()
        self.hdnSize = hdnSize
        self.embedding, numEmbeddings, embeddingDim = create_emb_layer(weightsMatrix, True)
        #self.embedding = nn.Embedding(input_size, hdnSize)
        self.gru = nn.GRU(hdnSize, hdnSize)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hdnSize, device=cuda)

class DecoderRNN(nn.Module):
  def __init__(self, hdnSize, output_size):
    super(DecoderRNN, self).__init__()
    self.hdnSize = hdnSize
    self.embedding = nn.Embedding(output_size, hdnSize)
    self.gru = nn.GRU(hdnSize, hdnSize)
    self.out = nn.Linear(hdnSize, output_size)
    self.softmax = nn.LogSoftmax(dim=1)
def forward(self, input, hidden):
    output = self.embedding(input).view(1, 1, -1)
    output = F.relu(output)
    output, hidden = self.gru(output, hidden)
    output = self.softmax(self.out(output[0]))
    return output, hidden
def initHidden(self):
    return torch.zeros(1, 1, self.hdnSize, device=cuda)

In [ ]:
class AttnDecoderRNN(nn.Module):
  def __init__(self, hdnSize, output_size, dropout_p=0.1, max_length=MX_LENGTH):
    super(AttnDecoderRNN, self).__init__()
    self.hdnSize = hdnSize
    self.output_size = output_size
    self.dropout_p = dropout_p
    self.max_length = max_length

    self.embedding = nn.Embedding(self.output_size, self.hdnSize)
    self.attn = nn.Linear(self.hdnSize * 2, self.max_length)
    self.attn_combine = nn.Linear(self.hdnSize * 2, self.hdnSize)
    self.dropout = nn.Dropout(self.dropout_p)
    self.gru = nn.GRU(self.hdnSize, self.hdnSize)
    self.out = nn.Linear(self.hdnSize, self.output_size)

  def forward(self, input, hidden, encoderOutputs):
      embedded = self.embedding(input).view(1, 1, -1)
      embedded = self.dropout(embedded)

      attn_weights = F.softmax(
        self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
      attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoderOutputs.unsqueeze(0))

      output = torch.cat((embedded[0], attn_applied[0]), 1)
      output = self.attn_combine(output).unsqueeze(0)

      output = F.relu(output)
      output, hidden = self.gru(output, hidden)

      output = F.log_softmax(self.out(output[0]), dim=1)
      return output, hidden, attn_weights

  def initHidden(self):
      return torch.zeros(1, 1, self.hdnSize, device=cuda)

In [ ]:
def indexesFromSentence(lang, sentence):
  return [lang.wordToIndex[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
  indexes = indexesFromSentence(lang, sentence)
  indexes.append(EOS_tkn)
  return torch.tensor(indexes, dtype=torch.long, device=cuda).view(-1, 1)

def tensorsFromPair(pair):
  inpTensor = tensorFromSentence(inp_lang, pair[0])
  target_tensor = tensorFromSentence(op_lang, pair[1])
  return (inpTensor, target_tensor)

In [ ]:
teacher_forcing_ratio = 0.5

In [ ]:
def train(inpTensor, target_tensor, encoder, decoder, encoderOptimizer, decoderOptimizer, criterion, max_length=MX_LENGTH):
  encoderHdn = encoder.initHidden()

  encoderOptimizer.zero_grad()
  decoderOptimizer.zero_grad()
  input_length = inpTensor.size(0)
  target_length = target_tensor.size(0)
  encoderOutputs = torch.zeros(max_length, encoder.hdnSize, device=cuda)
  loss = 0

  for ei in range(input_length):
    encoder_output, encoderHdn = encoder(
      inpTensor[ei], encoderHdn)
    encoderOutputs[ei] = encoder_output[0, 0]

  decoder_input = torch.tensor([[SOS_tkn]], device=cuda)
  decoder_hidden = encoderHdn
  use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

  if use_teacher_forcing:
    # Teacher forcing: Feed the target as the next input
    for di in range(target_length):
      decoderOpt, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoderOutputs)
      loss += criterion(decoderOpt, target_tensor[di])
      decoder_input = target_tensor[di] # Teacher forcing
  else:
    # Without teacher forcing: use its own predictions as the next input
    for di in range(target_length):
      decoderOpt, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoderOutputs)
      topv, topi = decoderOpt.topk(1)
      decoder_input = topi.squeeze().detach() # detach from history as input

      loss += criterion(decoderOpt, target_tensor[di])
      if decoder_input.item() == EOS_tkn:
        break
  loss.backward()

  encoderOptimizer.step()
  decoderOptimizer.step()

  return loss.item() / target_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plotLossTotal = 0  # Reset every plot_every

    encoderOptimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoderOptimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        inpTensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(inpTensor, target_tensor, encoder,
                     decoder, encoderOptimizer, decoderOptimizer, criterion)
        print_loss_total += loss
        plotLossTotal += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plotLossTotal / plot_every
            plot_losses.append(plot_loss_avg)
            plotLossTotal = 0

    showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
  plt.figure()
  fig, ax = plt.subplots()
  # this locator puts ticks at regular intervals
  loc = ticker.MultipleLocator(base=0.2)
  ax.yaxis.set_major_locator(loc)
  plt.plot(points)

def evaluate(encoder, decoder, sentence, max_length=MX_LENGTH):
  with torch.no_grad():
    inpTensor = tensorFromSentence(inp_lang, sentence)
    input_length = inpTensor.size()[0]
    encoderHdn = encoder.initHidden()

    encoderOutputs = torch.zeros(max_length, encoder.hdnSize, device=cuda)

    for ei in range(input_length):
      encoder_output, encoderHdn = encoder(inpTensor[ei], encoderHdn)
      encoderOutputs[ei] += encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_tkn]], device=cuda) # SOS

    decoder_hidden = encoderHdn

    decoded_words = []
    decoderAttentions = torch.zeros(max_length, max_length)

    for di in range(max_length):
      decoderOpt, decoder_hidden, decoder_attention = decoder(
        decoder_input, decoder_hidden, encoderOutputs)
      decoderAttentions[di] = decoder_attention.data
      topv, topi = decoderOpt.data.topk(1)
      if topi.item() == EOS_tkn:
        decoded_words.append('<EOS>')
        break
      else:
        decoded_words.append(op_lang.indexToword[topi.item()])

      decoder_input = topi.squeeze().detach()

    return decoded_words, decoderAttentions[:di + 1]

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hdnSize = 100
encoder1 = EncoderRNN(inp_lang.min_words, hdnSize).to(cuda)
attn_decoder1 = AttnDecoderRNN(hdnSize, op_lang.min_words, dropout_p=0.1).to(cuda)

trainIters(encoder1, attn_decoder1, 50000, print_every=10000)

2m 52s (- 11m 30s) (10000 20%) 5.0839
5m 44s (- 8m 37s) (20000 40%) 4.8676
8m 39s (- 5m 46s) (30000 60%) 4.7886
11m 33s (- 2m 53s) (40000 80%) 4.7361
14m 28s (- 0m 0s) (50000 100%) 4.7086


### Examples of English to Spanish Translations

In [ ]:
evaluateRandomly(encoder1, attn_decoder1)

> he hates air travel .
= odia los viajes en avion .
<  es es . . <EOS>

> can you tell me what this word means ?
=  puede decirme que significa esta palabra ?
<  no que de en . . . <EOS>

> he s friendly with all his classmates .
= el es amistoso con todos sus companeros .
<  no que de . . . . <EOS>

> she was crying in her room .
= ella estaba llorando en su habitacion .
<  no que de . . . <EOS>

> she complained to him about everything .
= se quejaba de todo con el .
<  no que de . . . <EOS>

> what does tom think about this ?
=  que opina tom de esto ?
<  no que de . . . <EOS>

> you have a great job .
= tienes un buen trabajo .
<  es que . . . <EOS>

> tom can run fast .
= tom puede correr rapido .
<  es es . . <EOS>

> she spends a lot of money on shoes .
= ella gasta mucho dinero en zapatos .
<  no que de en el . . <EOS>

> they never came .
= ellos nunca vinieron .
<  es . . <EOS>



#We now translate the sentences from Part 1

In [ ]:
def translate(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))

In [ ]:
translate('she had a an hat')

input = she had a an hat
output =  es es . . <EOS>


In [ ]:
translate('how they re you')

input = how they re you
output =  es . . <EOS>


In [ ]:
translate('look the the the')

input = look the the the
output =  es . . <EOS>


In [ ]:
translate('i m good')

input = i m good
output =  es . <EOS>


In [ ]:
translate('i am a big person')

input = i am a big person
output =  es es . . <EOS>


## Here, We observed in the final translation for the language conversion eng to esp more training is needed to be done with a lot more training words since in the output only a few words are getting detected